In [50]:
from helper import print_sents, accuracy, tokenized_word_sets

**tf.tsv**

always predict A-coref as True and B-coref as False

In [51]:
f_test = open("gap-test.tsv", "r")
result = open("tf.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    result.write("\t".join([attrs[0]] + ['True', 'False']) + "\n")

f_test.close()
result.close()

In [52]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=tf.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 51.8 precision: 45.9 f1: 48.7
		tp 918	fp 1082
		fn 855	tn 1145
Masculine recall: 51.0 precision: 45.3 f1: 48.0
		tp 453	fp 547
		fn 436	tn 564
Feminine recall: 52.6 precision: 46.5 f1: 49.4
		tp 465	fp 535
		fn 419	tn 581
Bias (F/M): 1.03



**closer.tsv**

Predict closer coreference to pronoun as True

In [53]:
f_test = open("gap-test.tsv", "r")
result = open("closer.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    tf = ['True', 'False']
    if abs(int(attrs[3]) - int(attrs[5])) > abs(int(attrs[3]) - int(attrs[8])):
        tf = ['False', 'True']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [54]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=closer.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 50.1 precision: 44.4 f1: 47.1
		tp 888	fp 1112
		fn 885	tn 1115
Masculine recall: 51.6 precision: 45.9 f1: 48.6
		tp 459	fp 541
		fn 430	tn 570
Feminine recall: 48.5 precision: 42.9 f1: 45.5
		tp 429	fp 571
		fn 455	tn 545
Bias (F/M): 0.94



In [55]:
print_sents(random=True)

She became nationally famous following her 1923 victory in the U.S. Women's Amateur. On August 25, 1924, she became the first golfer and first female athlete to appear on the cover of Time magazine. Edith Cummings was born on March 26, 1899 to David Cummings and Ruth Dexter. She had a younger brother, Dexter.

Ferrari also cast doubt on ``the arrival of technicians'' with Alonso. At the first race in Bahrain, Alonso qualified third behind team mate Massa and pole sitter Sebastian Vettel. At the start, Alonso got ahead of Massa to move up to second and later on in the race, leader Vettel had an engine problem and dropped to fourth and Alonso won the race, becoming the fifth man to win on his debut for Ferrari after Juan Manuel Fangio in 1956, Mario Andretti in 1971, Nigel Mansell in 1989, and Kimi R*ikk*nen in 2007.

She begins a relationship with him, unaware he is also seeing Gloria Todd (Sue Jenkins), a barmaid at the Rovers Return Inn. When Rita discovers his infidelity, she issues 

In [56]:
import pandas as pd

In [57]:
train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data = pd.read_csv('gap-test.tsv', sep='\t')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2000 non-null   object
 1   Text            2000 non-null   object
 2   Pronoun         2000 non-null   object
 3   Pronoun-offset  2000 non-null   int64 
 4   A               2000 non-null   object
 5   A-offset        2000 non-null   int64 
 6   A-coref         2000 non-null   bool  
 7   B               2000 non-null   object
 8   B-offset        2000 non-null   int64 
 9   B-coref         2000 non-null   bool  
 10  URL             2000 non-null   object
dtypes: bool(2), int64(3), object(6)
memory usage: 144.7+ KB


**subject.tsv**

Predict farther coreference pronoun as True when pronoun is subject pronoun (he, she).
Predict closer coreference pronoun as True when pronoun is not subject pronoun, which is object or possessive pronoun(her, his, him).

In [58]:
pronouns = set(train_data['Pronoun'])
pronouns

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [59]:
result = open("subject.tsv", 'w')

for attrs in test_data.iloc:
    pro, pro_id = attrs[2:4]
    A, A_id = attrs[4:6]
    B, B_id = attrs[7:9]

    if pro.lower() in ['he', 'she']:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['TRUE', 'FALSE']
        else:
            tf = ['FALSE', 'TRUE']
    else:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['FALSE', 'TRUE']
        else:
            tf = ['TRUE', 'FALSE']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [60]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 58.3 precision: 51.6 f1: 54.8
		tp 1033	fp 967
		fn 740	tn 1260
Masculine recall: 59.4 precision: 52.8 f1: 55.9
		tp 528	fp 472
		fn 361	tn 639
Feminine recall: 57.1 precision: 50.5 f1: 53.6
		tp 505	fp 495
		fn 379	tn 621
Bias (F/M): 0.96



In [61]:
false_case = 0
for attrs in test_data.iloc:
    if attrs[6] == attrs[9]:
        false_case += 1

false_case/len(test_data)

0.1135

# subject.tsv

draw a relation between two proper nouns with sentence structure

(e.g. Which one is the subject?)

In [62]:
import nltk
sent = "In the next pair, Mikhaylov equalled Grishin's time and they shared Olympic gold."
tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent))

Classify train_data into several categories

1. When words are inside parenthesis
2. When words appear after the pronoun
 * When the pronoun is subject pronoun
 * When the pronoun is object/possessive pronoun
3. When the both coreferences are false
4. When only one word is subject
5. When the both words are subject
6. When none of the words are subject
    

In [63]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')

## 1. When words are inside parenthesis
Assume when word is inside parenthesis, coreference is always false

In [64]:
parenthesis_data = []

def isparenthesis(sent, word, offset):
    if sent[offset-1] == "(" and sent[offset+len(word)] == ")":
        return True
    else:
        return False

for row in train_data.iloc:
    if isparenthesis(row['Text'], row['A'], row['A-offset']) or \
       isparenthesis(row['Text'], row['B'], row['B-offset']):
            parenthesis_data.append(row)

In [65]:
parenthesis_data = pd.DataFrame(parenthesis_data, columns=train_data.columns)

In [66]:
print_sents(datas={'manual':True, 'data':parenthesis_data}, num_lines = 10)

Alf and Else's daughter Rita (Una Stubbs) now lives with her husband Mike in his hometown of Liverpool and often visits her parents, although Mike does not appear (as Antony Booth had no interest in reprising the role). Usually, Alf is seen drinking with his friend Arthur (Arthur English) in the local pub. Although his beloved Conservative Party has returned to power, Alf is not happy with Margaret Thatcher being Prime Minister because, according to him, ``a woman's place is in the home''.

Liam (Roberto Aguire) surprises Aria (Lucy Hale) in Rosewood and Aria admits to Liam the truth about Ezra's new book after he reveals to her the writing is too similar to her own. Alison (Sasha Pieterse) explains to Spencer that she and Elliott (Huw Collins) are more than just friends, but they cannot reveal their relationship as it would be deemed ``inappropriate''. Spencer confronts Mona (Janel Parrish), who tells her that she had Yvonne leave her phone on purpose because she wanted Spencer to fin

result : 22 out of 31 sentences are predicted correct --> accuracy : 71%

## 2. When words appear after the pronoun
Assume coreference of words coming after the pronoun is false

In [67]:
after_pronoun = []

for row in train_data.iloc:
    if row['Pronoun-offset'] < row['A-offset'] or row['Pronoun-offset'] < row['B-offset']:
        after_pronoun.append(row)

after_pronoun = pd.DataFrame(after_pronoun, columns=train_data.columns)

In [68]:
after_subject_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['he', 'she', 'He', 'She'])]
after_other_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['his', 'her', 'him', 'His', 'Her', 'Him'])]

In [69]:
print_sents(datas={'manual':True, 'data':after_subject_pronoun}, num_lines = 10)

Isabel Walker is a twenty-something American film-school dropout who decides to visit her step-sister Roxanne, a poet who now lives in France. Isabel believes that she is there to help Roxanne during her pregnancy with her toddler infant, but later realizes that her father and step-mother sent her there so that Roxanne would help the shiftless Isabel gain some direction in life. Shortly after she arrives, Roxanne confides in Isabel that her French husband, Claude-Henri has left her.

Yuan Sijun (born 29 May 2000 in Fujian) is a Chinese snooker player. Yuan started to play snooker at the age of 10 and first drew attention in 2016 when, at the age of 15, he defeated world number 13 Martin Gould in a 5--0 whitewash in the opening round of the 2016 China Open before losing 5--2 to Graeme Dott. In 2015, Yuan entered the 2015 ACBS Asian Under-21 Snooker Championship where he reached the final, but lost eventually to Akani Songsermsawad.

The audience feels the effects of the previous notes a

In [70]:
def after_pronoun_predict(data):
    A_pred = []
    B_pred = []
    for row in data.iloc:
        A_value = False if row['A-offset'] > row['Pronoun-offset'] else True
        B_value = False if row['B-offset'] > row['Pronoun-offset'] else True

        A_pred.append(A_value)
        B_pred.append(B_value)

    data['A-pred'] = A_pred
    data['B-pred'] = B_pred

In [71]:
after_pronoun_predict(after_subject_pronoun)
after_pronoun_predict(after_other_pronoun)

<ipython-input-70-ceb8dec2fe17>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['A-pred'] = A_pred
<ipython-input-70-ceb8dec2fe17>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['B-pred'] = B_pred


In [72]:
print(accuracy(after_subject_pronoun))
print(accuracy(after_other_pronoun))

0.411214953271028
0.2133676092544987


Result : It doesn't really matter whether a word comes after the pronoun or not

## 3. When the both word coreference is false

Let's find out what they have in common.

In [73]:
false_cases = 0
for row in train_data.iloc:
    if row['A-coref'] == row['B-coref']:
        false_cases += 1

print(false_cases/len(train_data))
print(false_cases)

0.1005
201


There are more than 10 percent of these kind of cases

In [ ]:
falses_data = train_data[(train_data['A-coref'] == False) & (train_data['B-coref'] == False)]

In [27]:
print_sents(datas={'manual':True, 'data':falses_data}, num_lines = 10)

Licence to Kill is one of the best of the recent Bonds.'' Jack Kroll, writing in Newsweek described Licence to Kill as ``a pure, rousingly entertaining action movie''. Kroll was mixed in his appraisal of Dalton, calling him ``a fine actor who hasn't yet stamped Bond with his own personality'', observing ``Director John Glen is the Busby Berkeley of action flicks, and his chorus line is the legendary team of Bond stunt-persons who are at their death-defying best here''.

Many middle- and back-markers also made significant gains in the first lap, especially Verstappen, who overtook nine cars in the space of two minutes earning the title of ``Driver of the Day''. The early action was however soon neutralised following contact between P*rez and Stroll which left Stroll spun out and beached in a gravel trap with a ruined set of tyres. The removal of his car warranted a virtual safety car during which Vettel pitted for slicks.

Her name, which means ``My friend'' in (French: mon ami), is giv

## 4. When only one word is subject
Assume that subject word is always true, and other word is always false

In [28]:
subject_data = train_data[train_data['Pronoun'].isin(['he', 'she', 'He', 'She'])]
other_data = train_data[train_data['Pronoun'].isin(['his', 'hin', 'her', 'His', 'Him', 'Her'])]
print(len(subject_data))
print(len(other_data))

801
1101


In [32]:
def word_subject(row):
    raw = row['Text']
    words, tagged_words, word_index = tokenized_word_sets(row)

    A_id = row['A-offset']
    B_id = row['B-offset']

    def next_word(word):

        tokens = len(row[word].split())
        id = row[word+'-offset']
        index = word_index[id] + tokens

        if tagged_words[index] == ('(', '('):
            close = tagged_words[index:].index((')', ')'))
            if index + close + 1 >= len(tagged_words):
                return tagged_words[index]
            return tagged_words[index + close + 1]
        
        return tagged_words[index]

    if next_word('A')[1].startswith('V'):
        if next_word('B')[1].startswith('V'):
            return (True, True)
        else:
            return (True, False)
    else:
        if next_word('B')[1].startswith('V'):
            return (False, True)
        else:
            return (False, False)

def only_subject(row):
    return sum(word_subject(row))%2 == 1

In [82]:
only_subject_data = train_data.apply(only_subject, axis=1)
only_subject_data = train_data[only_subject_data]
len(only_subject_data)

1088

In [83]:
only_subject_data['A-pred'] = only_subject_data.apply(lambda row: word_subject(row)[0], axis=1)
only_subject_data['B-pred'] = only_subject_data.apply(lambda row: word_subject(row)[1], axis=1)

<ipython-input-83-146d5bf28ab4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_subject_data['A-pred'] = only_subject_data.apply(lambda row: word_subject(row)[0], axis=1)
<ipython-input-83-146d5bf28ab4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_subject_data['B-pred'] = only_subject_data.apply(lambda row: word_subject(row)[1], axis=1)


In [84]:
accuracy(only_subject_data)

0.7398897058823529

## 5. When both words are subject
Assume that latter word is True, while the other is False

In [90]:
def former_word(row):
    if row['A-offset'] < row['B-offset']:
        return (True, False)
    return (False, True)

def latter_word(row):
    if row['A-offset'] < row['B-offset']:
        return (False, True)
    return (True, False)

In [36]:
def both_subject(row):
    return word_subject(row) == (True, True)

In [85]:
both_subject_data = train_data.apply(both_subject, axis=1)
both_subject_data = train_data[both_subject_data]
len(both_subject_data)

172

In [87]:
print_sents(datas={'manual':True, 'data':both_subject_data}, num_lines = 10)

In 1947 McCrae persuaded Bradley to run professionally, where he had his first run in the Powderhall sprint, winning his heat and attracting the attention of bookmaker William Murphy who offered to sponsor Bradley for a preparation at the famous New year sprint. In 1951, Bradley became a training partner for Eric Cumming, the champion Australian professional athlete who was on a 10 week preparation for the Powderhall sprint. Cumming was beaten in the semi final and was ready to return to Australia before Bradley convinced him to stay in Scotland and have another go at Powderhall in 1952.

She also has fun tormenting Betty about her crush on Henry, calling him her ``Nerd Lover.'' In the episode ``Crush'd'' she becomes Betty's roommate and despite the ups and downs, both begin to bond slowly, Amanda taking the responsibility of paying half of the rent and Betty starting to take her advice seriously.

He directed the White House Climate Change Task Force and spent time as a senior member 

In [91]:
both_subject_data['A-pred'] = both_subject_data.apply(lambda row: latter_word(row)[0], axis=1)
both_subject_data['B-pred'] = both_subject_data.apply(lambda row: latter_word(row)[1], axis=1)

<ipython-input-91-b3ccc41bfd35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both_subject_data['A-pred'] = both_subject_data.apply(lambda row: latter_word(row)[0], axis=1)
<ipython-input-91-b3ccc41bfd35>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both_subject_data['B-pred'] = both_subject_data.apply(lambda row: latter_word(row)[1], axis=1)


In [92]:
accuracy(both_subject_data)

0.5988372093023255

### * And when both words are not subject

In [39]:
def none_subject(row):
    return word_subject(row) == (False, False)

In [40]:
none_subject_data = subject_data.apply(none_subject, axis=1)
none_subject_data = subject_data[none_subject_data]
len(none_subject_data)

285

In [95]:
print_sents(datas={'manual':True, 'data':none_subject_data}, num_lines = 10)

As described by Brittany Spanos of Rolling Stone, ``The video opens on an art gallery filled with photos of Bieber's silhouette before shifting to the singer offering an emotional delivery of the track in a dark room. Bieber eventually becomes covered in animated paint, drawings and caricatures. Back in the gallery, Jack * fans are providing the illustrations that are appearing over or surrounding Bieber's body as he sings and dances to the track.''

In the summer of 2012, Maicon joined Shakhtar Donetsk on a free transfer. He signed a three-year contract. On 4 September it was announced that Maicon would join Zorya on loan for six months before joining up with Shakhtar again during the winter break. Because he joined Shakhtar after the season had begun Mircea Lucescu, the Shakhtar manager, decided to loan him to Zorya for the first half of the season.

Finally, Tish marks April 29 Again, Really April 30 Very Early in her journal ``DO read this, Mrs. Dunphrey.'' In this last entry, she 

In [93]:
none_subject_data['A-pred'] = none_subject_data.apply(lambda row: former_word(row)[0], axis=1)
none_subject_data['B-pred'] = none_subject_data.apply(lambda row: former_word(row)[1], axis=1)

<ipython-input-93-d76347394a96>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  none_subject_data['A-pred'] = none_subject_data.apply(lambda row: former_word(row)[0], axis=1)
<ipython-input-93-d76347394a96>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  none_subject_data['B-pred'] = none_subject_data.apply(lambda row: former_word(row)[1], axis=1)


In [94]:
accuracy(none_subject_data)

0.44912280701754387

## 5. When the pronoun is object/possessive pronoun

### * And when all words are in different sentence with the pronoun

Assume subject words are true, other is false

In [42]:
def different_sents(row):
    raw = row['Text']
    words, tagged_words, word_index = tokenized_word_sets(row)

    dots = [i for i, w in enumerate(words) if w == '.']

    pro_id = word_index[row['Pronoun-offset']]
    A_id = word_index[row['A-offset']]
    B_id = word_index[row['B-offset']]

    def between(id1, id2):
        if id1 > id2:
            id1, id2 = id2, id1
        dots = [i for i, w in enumerate(words[id1:id2]) if w == '.']
        if len(dots) > 0:
            return True
        else:
            return False

    return (between(pro_id, A_id), between(pro_id, B_id))

def both_different_sents(row):
    return different_sents(row) == (True, True)
    

In [43]:
both_different = other_data.apply(both_different_sents, axis=1)
both_different = other_data[both_different]
len(both_different)

205

In [44]:
print_sents(datas={'manual':True, 'data':both_different}, num_lines = 10)

Shortly after announcing she would be candidate, a poll for the RTL 4 News estimated that 56% of the people preferred Verdonk as the leader of the VVD. Rita Verdonk was advised by Kay van de Linde, who also played a part in anti-immigration politician Pim Fortuyn 's 2002 election campaign, which was cut short by his assassination on May 6. Several of her statements and policies seem to exploit xenophobic emotions in the country, according to her opponents.

Violet Winifred Leslie Henry Anderson was from Blairgowrie, Perthshire, the daughter of Isaac Henry-Anderson and Katherine Blyth. Her father was a solicitor and secretary of the Blairgowrie Golf Club before he went bankrupt in 1904 and left his family.

Her triumvirate of conductors--Levi, Richter and Felix Mottl--shared the musical direction until 1894, when Levi left. Richter and Mottl served throughout Cosima's years, joined by several of the leading conductors of the day, although B*low resisted all offers to participate. In the

In [45]:
def only_different_sents(row):
    return sum(different_sents(row)) == 1
    
only_different = other_data.apply(only_different_sents, axis=1)
only_different = other_data[only_different]
len(only_different)

185

In [46]:
print_sents(datas={'manual':True, 'data':only_different}, num_lines = 10)

Clay's group was chosen to maintain electronic equipment during the planned invasion of Japan. After separation from the Army, Clay completed both the Bachelor's (1947) and Masters (1948) degrees in Physics at Kansas State University. At the University of Wisconsin-Madison, his doctoral dissertation under Professor Gibson Winas was entitled, ``Field Strengths and Spectra of High Frequency Gas Discharges.''

The exhibition will display works of Mr. Kwan, many are shown on a loan basis with the kind consent of galleries and friends, as well as related articles, photos and information. It will not only provide a broad view of the creations of Mr. Kwan, a core figure of our art scene, but also an opportunity to look at the life of an enthusiastic and committed artist. Dedicated to his life to art making, Jerry used oil painting as his major medium of expression in his artwork.

Carl Alexander Clerck (1709 -- 22 July 1765) was a Swedish entomologist and arachnologist. Clerck came from a fam

In [77]:
only_other_data = other_data.apply(only_subject, axis=1)
only_other_data = other_data[only_other_data]
len(only_other_data)

574

In [78]:
only_other_data['A-pred'] = only_other_data.apply(lambda row: word_subject(row)[0], axis=1)
only_other_data['B-pred'] = only_other_data.apply(lambda row: word_subject(row)[1], axis=1)

<ipython-input-78-5dc47d9aa78e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_other_data['A-pred'] = only_other_data.apply(lambda row: word_subject(row)[0], axis=1)
<ipython-input-78-5dc47d9aa78e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_other_data['B-pred'] = only_other_data.apply(lambda row: word_subject(row)[1], axis=1)


In [79]:
accuracy(only_other_data)

0.7003484320557491

In [80]:
both_other_data = other_data.apply(both_subject, axis=1)
both_other_data = other_data[both_other_data]
len(both_other_data)

100

In [81]:
print_sents(datas={'manual':True, 'data':both_other_data}, num_lines = 10)

He brings her to his small hometown, where she is not exactly greeted with open arms. Hubbell's overbearing mother, Fanny, is appalled by the news of their marriage and frowns upon the idea of Michelle's moving in with them. Fanny hosts a wedding reception, where Michelle interacts with the colorful locals, who all seem to despise her.

The next day, Peggy visits Stacey in hospital and asks her to confess to burning the pub, as a sentence for arson would be less than one for murder, but Stacey refuses. Peggy realises that Stacey needs to look after her daughter so she leaves.

Suspecting Cilla was having another affair, Les kicked her out and Cilla came clean to Fiz about her illness, who told Chesney and Les. The family were reunited by Cilla's hospital bed, and Cilla then returned home. On finding out that husband Les and best mate Yana had slept together, Cilla hatched a plan to get her own back.

By the time the new councilmembers were sworn in, the Robles majority on the City Coun